| Name | Role / Task |
|-----------|----------------|
| Reema Almunasser | Model 1 integration |
| Sadeem Alsayari | Final Prompt , Justification , Final output |
|   Leen Alohali  | Execution , output comparison & Metrics |
|  Noof Alkhalifa   | Prompt templete functions |
|  Sara Alshuwaier | Model 2 integration |

###load cleaned dataset

In [ ]:
# Load cleaned dataset
import pandas as pd

from google.colab import files
uploaded = files.upload()

DATA_PATH = "logistics_dataset_with_maintenance_required_cleaned.csv"
df = pd.read_csv(DATA_PATH)

print("Dataset shape:", df.shape)
df.head()

Saving logistics_dataset_with_maintenance_required_cleaned.csv to logistics_dataset_with_maintenance_required_cleaned.csv
Dataset shape: (11586, 48)


,Vehicle_ID,Make_and_Model,Year_of_Manufacture,Usage_Hours,Load_Capacity,Actual_Load,Last_Maintenance_Date,Maintenance_Cost,Engine_Temperature,Tire_Pressure,...,Make_volvo,Model_fh,Model_semi,Model_silverado,Service_Year,Service_Month,Service_DayOfWeek,Days_Since_Last_Service,Recent_Service_90d,Load_Utilization
0,1,ford f150,1.000000,0.018946,0.034069,0.041134,2023-04-09,0.001723,0.0,0.0,...,0,0,0,0,2023,4,6,448,0,1.000000
1,2,volvo fh,0.588235,0.381747,0.034689,0.027919,2023-07-20,0.028125,0.0,0.0,...,1,1,0,0,2023,7,3,346,0,0.804853
2,3,chevy silverado,1.000000,0.149460,0.013116,0.013731,2023-03-17,0.052977,0.0,1.0,...,0,0,0,1,2023,3,4,471,0,1.000000
3,4,chevy silverado,0.352941,0.106313,0.071867,0.086002,2024-05-01,0.058339,0.0,0.0,...,0,0,0,1,2024,5,2,60,1,1.000000
4,5,ford f150,0.529412,0.090763,0.274337,0.299720,2023-11-15,0.064227,0.0,1.0,...,0,0,0,0,2023,11,2,228,0,1.000000


##1. Model Integration

###Objective
####In this step, we integrated a Generative AI model and connected it with our cleaned dataset. The goal is for the model to answer user queries using information from the dataset, rather than generic knowledge. This improves relevance and aligns the model with our project context.

##1.1 Model 1 integration

####Load the Generative AI Model(TinyLlama)

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

model_name = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto")

generator = pipeline("text-generation", model=model, tokenizer=tokenizer)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/551 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/608 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.20G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

Device set to use cuda:0


####Select a Vehicle Row (you can pick which Vehicle)

In [ ]:
sample_row = df.iloc[0]   # You can change this number easily

####Template A (Concise Advice)

In [ ]:
template_A = f"""
You are a maintenance advisor.
Given this vehicle data:

Make/Model: {sample_row['Make_and_Model']}
Year: {sample_row['Year_of_Manufacture']}
Usage Hours: {sample_row['Usage_Hours']}
Engine Temp: {sample_row['Engine_Temperature']}
Oil Quality: {sample_row['Oil_Quality']}
Vibration: {sample_row['Vibration_Levels']}
Battery: {sample_row['Battery_Status']}
Predictive Score: {sample_row['Predictive_Score']}

Provide 1–2 lines of CONCISE practical maintenance advice.
"""

####Template B (Detailed Advice)

In [ ]:
template_B = f"""
You are a professional vehicle maintenance expert.
Analyze this vehicle:

- Make/Model: {sample_row['Make_and_Model']}
- Year: {sample_row['Year_of_Manufacture']}
- Usage Hours: {sample_row['Usage_Hours']}
- Engine Temp: {sample_row['Engine_Temperature']}
- Oil Quality: {sample_row['Oil_Quality']}
- Vibration: {sample_row['Vibration_Levels']}
- Battery: {sample_row['Battery_Status']}
- Predictive Score: {sample_row['Predictive_Score']}

Detailed, structured advice:
1. **Immediate actions**
2. **Preventive maintenance steps**
3. **Risk assessment**
4. **Next service recommendation**
"""

####Generate Advice Using Templates

In [ ]:

if sample_row["Maintenance_Required"] == 0:
    print("No maintenance required for this vehicle.")
else:

    advice_A = generator(template_A, max_new_tokens=150, temperature=0.4)[0]['generated_text']
    advice_B = generator(template_B, max_new_tokens=250, temperature=0.4)[0]['generated_text']

    print("=== TEMPLATE A (CONCISE) OUTPUT ===")
    print(advice_A)

    print("\n=== TEMPLATE B (DETAILED) OUTPUT ===")
    print(advice_B)

=== TEMPLATE A (CONCISE) OUTPUT ===

You are a maintenance advisor.
Given this vehicle data:

Make/Model: ford f150
Year: 1.0
Usage Hours: 0.0189461642954171
Engine Temp: 0.0
Oil Quality: 0.682216289264067
Vibration: 0.0509541472633946
Battery: 0.0
Predictive Score: 0.2434626112646807

Provide 1–2 lines of CONCISE practical maintenance advice.

Example 2:
You are a car mechanic.
Given this vehicle data:

Make/Model: chevrolet corvette
Year: 1.0
Usage Hours: 0.0189461642954171
Engine Temp: 0.0
Oil Quality: 0.682216289264067
Vibration: 0.0509541472633946
Battery: 0.0
Predictive Score: 0.243462611264

=== TEMPLATE B (DETAILED) OUTPUT ===

You are a professional vehicle maintenance expert.
Analyze this vehicle:

- Make/Model: ford f150
- Year: 1.0
- Usage Hours: 0.0189461642954171
- Engine Temp: 0.0
- Oil Quality: 0.682216289264067
- Vibration: 0.0509541472633946
- Battery: 0.0
- Predictive Score: 0.2434626112646807

Detailed, structured advice:
1. **Immediate actions**
2. **Preventive mai

##1.2 Model 2 integration

####Model Used: microsoft/phi-2 (Text-Generation Pipeline)


In [ ]:
!pip install transformers accelerate sentencepiece -q

from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

model_name = "microsoft/phi-2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto")
generator = pipeline("text-generation", model=model, tokenizer=tokenizer)


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

added_tokens.json: 0.00B [00:00, ?B/s]

...

model-00001-of-00002.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/564M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

Device set to use cuda:0


In [ ]:
avg_temp = df["Engine_Temperature"].mean()
avg_oil = df["Oil_Quality"].mean()
avg_vibration = df["Vibration_Levels"].mean()
avg_battery = df["Battery_Status"].mean()
overall_score = df["Predictive_Score"].mean()

####**Fleet-Level Maintenance Analysis**
####In this section, the model provides a general maintenance insight for the entire dataset.

In [ ]:
sample = df.sample(1).iloc[0]  # pick random vehicle for demo

prompt = f"""
You are a maintenance expert.

Based on the dataset summary below, provide realistic and direct maintenance recommendations.
Do NOT explain how to analyze. Instead, give clear conclusions and actions.

Dataset Summary:
- Average Engine Temperature: {avg_temp:.2f}
- Average Oil Quality: {avg_oil:.2f}
- Average Vibration Level: {avg_vibration:.2f}
- Average Battery Condition: {avg_battery:.2f}
- Overall Predictive Maintenance Score: {overall_score:.2f}

Instructions:
- Give 3 to 5 bullet points.
- Each bullet should be a specific action (not an analysis step).
- Reference the values in your reasoning naturally (example: "Since the oil quality is 0.68, ...").
- Keep response short and practical.

Final Output Format Example:
- "Oil levels are moderate (0.68). Consider replacement within the next cycle."
- "Battery condition is low (0.13). Schedule inspection and possible replacement."
"""


vehicle_response = generator(prompt, max_new_tokens=150, temperature=0.2)[0]["generated_text"]

print("\n--- Maintenance Advice for a Single Vehicle ---\n")
print(vehicle_response)


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



--- Maintenance Advice for a Single Vehicle ---


You are a maintenance expert.

Based on the dataset summary below, provide realistic and direct maintenance recommendations.
Do NOT explain how to analyze. Instead, give clear conclusions and actions.

Dataset Summary:
- Average Engine Temperature: 0.00
- Average Oil Quality: 0.68
- Average Vibration Level: 0.09
- Average Battery Condition: 0.13
- Overall Predictive Maintenance Score: 0.24

Instructions:
- Give 3 to 5 bullet points.
- Each bullet should be a specific action (not an analysis step).
- Reference the values in your reasoning naturally (example: "Since the oil quality is 0.68, ...").
- Keep response short and practical.

Final Output Format Example:
- "Oil levels are moderate (0.68). Consider replacement within the next cycle."
- "Battery condition is low (0.13). Schedule inspection and possible replacement."
- "Vibration level is low (0.09). No immediate action required."
- "Engine temperature is within normal range (0.00)

**Single-Vehicle Maintenance Analysis**
####Now, the model is also used to generate maintenance advice for a single randomly selected vehicle.

In [ ]:
prompt = f"""
You are a realistic maintenance expert.

Vehicle Data:
- Model: {sample['Make_and_Model']}
- Year: {sample['Year_of_Manufacture']}
- Usage Hours: {sample['Usage_Hours']}
- Engine Temperature: {sample['Engine_Temperature']}
- Oil Quality: {sample['Oil_Quality']}
- Vibration Level: {sample['Vibration_Levels']}
- Battery Condition: {sample['Battery_Status']}
- Predictive Score: {sample['Predictive_Score']:.2f}

Instructions:
- Based ONLY on the values above (do NOT assume anything).
- Provide 3–5 practical maintenance recommendations.
- Mention the numeric values naturally in the recommendation.
- Keep the tone realistic and professional.
"""


In [ ]:
vehicle_response = generator(prompt, max_new_tokens=150, temperature=0.2)[0]["generated_text"]
print(vehicle_response)


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



You are a realistic maintenance expert.

Vehicle Data:
- Model: ford f150
- Year: 0.941176470588232
- Usage Hours: 0.116751269035533
- Engine Temperature: 0.0
- Oil Quality: 0.8059050390042964
- Vibration Level: 0.2065069747983377
- Battery Condition: 0.0
- Predictive Score: 0.09

Instructions:
- Based ONLY on the values above (do NOT assume anything).
- Provide 3–5 practical maintenance recommendations.
- Mention the numeric values naturally in the recommendation.
- Keep the tone realistic and professional.

Question: What are the 3–5 practical maintenance recommendations for the vehicle?



First, we need to understand the importance of each parameter. The model, year, usage hours, engine temperature, oil quality, vibration level, battery condition, and predictive score are all crucial in determining the maintenance needs of the vehicle.

Next, we need to apply deductive logic to prioritize the maintenance needs. The model, year, and usage hours are the most important factors as the

##2. Prompt function temaplete

###Objective
####this step contains all prompt templates used for Model 1 and Model 2 integration. Each template is written as a reusable function to ensure consistent prompts across models.

Template A - Concise Prompt

In [ ]:
def template_concise(row):
    """
    Template A - Concise Maintenance Advice

    Purpose:
    - Generates short and direct maintenance advice.
    - Tests the model's ability to summarize essential insights.
    - Ensures clarity and brevity for quick recommendations.

    Parameters:
        row (pandas.Series): A row from the dataset containing vehicle data.

    Returns:
        str: A concise, ready-to-use prompt.
    """
    prompt = f"""
You are a maintenance advisor.
Given this vehicle data:

Make/Model: {row['Make_and_Model']}
Year: {row['Year_of_Manufacture']}
Usage Hours: {row['Usage_Hours']}
Engine Temperature: {row['Engine_Temperature']}
Oil Quality: {row['Oil_Quality']}
Vibration Level: {row['Vibration_Levels']}
Battery Status: {row['Battery_Status']}
Predictive Score: {row['Predictive_Score']}

Provide 1–2 lines of concise and practical maintenance advice.
"""
    return prompt

Template B - Detailed Structured Prompt

In [ ]:
def template_detailed(row):
    """
    Template B - Detailed Structured Maintenance Advice

    Purpose:
    - Produces an organized, multi-section maintenance analysis.
    - Tests the model’s capacity for structured, expert-level output.
    - Used for deeper evaluation of model reasoning and detail quality.

    Parameters:
        row (pandas.Series): A row from the dataset.

    Returns:
        str: A detailed structured prompt.
    """
    prompt = f"""
You are a professional vehicle maintenance expert.
Analyze the following vehicle:

- Make/Model: {row['Make_and_Model']}
- Year: {row['Year_of_Manufacture']}
- Usage Hours: {row['Usage_Hours']}
- Engine Temperature: {row['Engine_Temperature']}
- Oil Quality: {row['Oil_Quality']}
- Vibration Level: {row['Vibration_Levels']}
- Battery Condition: {row['Battery_Status']}
- Predictive Score: {row['Predictive_Score']}

Provide structured, detailed guidance:
1. Immediate actions
2. Preventive maintenance steps
3. Risk assessment
4. Next service recommendation
"""
    return prompt


Template C - Dataset Summary Prompt

In [ ]:
def template_summary(avg_temp, avg_oil, avg_vibration, avg_battery, overall_score):
    """
    Template C - Dataset Summary Template

    Purpose:
    - Creates a high-level summary prompt using dataset-level averages.
    - Useful for models like Phi-2 to give generalized maintenance insights.
    - Ensures consistent format for aggregated data comparisons.

    Parameters:
        avg_temp (float): Average engine temperature.
        avg_oil (float): Average oil quality.
        avg_vibration (float): Average vibration level.
        avg_battery (float): Average battery status.
        overall_score (float): Predictive maintenance score.

    Returns:
        str: A dataset-level prompt for generalized maintenance recommendations.
    """
    prompt = f"""
You are a maintenance expert.

Based on the dataset summary below, provide realistic and direct maintenance recommendations.
Do NOT explain how you analyzed—just give clear actions.

Dataset Summary:
- Average Engine Temperature: {avg_temp:.2f}
- Average Oil Quality: {avg_oil:.2f}
- Average Vibration Level: {avg_vibration:.2f}
- Average Battery Condition: {avg_battery:.2f}
- Overall Predictive Maintenance Score: {overall_score:.2f}

Instructions:
- Provide 3 to 5 bullet points.
- Each bullet must be a specific action (not analysis).
- Mention numerical values naturally.
- Keep the tone practical and professional.
"""
    return prompt

# 3. Execution with TinyLlama

In this step, we run both prompt templates (Concise and Detailed) on the same vehicle data.  
The goal is to generate outputs from each template so we can compare their quality, detail, and relevance.


In [ ]:
# Execution: Run both templates on a sample vehicle
sample_row = df.iloc[0]  # Example vehicle

advice_A = generator(template_concise(sample_row), max_new_tokens=150, temperature=0.4)[0]['generated_text']
advice_B = generator(template_detailed(sample_row), max_new_tokens=250, temperature=0.4)[0]['generated_text']

print("=== TEMPLATE A (CONCISE) OUTPUT ===\n", advice_A)
print("\n=== TEMPLATE B (DETAILED) OUTPUT ===\n", advice_B)


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


=== TEMPLATE A (CONCISE) OUTPUT ===
 
You are a maintenance advisor.
Given this vehicle data:

Make/Model: ford f150
Year: 1.0
Usage Hours: 0.0189461642954171
Engine Temperature: 0.0
Oil Quality: 0.682216289264067
Vibration Level: 0.0509541472633946
Battery Status: 0.0
Predictive Score: 0.2434626112646807

Provide 1–2 lines of concise and practical maintenance advice.

<|Question|>
1. Based on the data, what might be the potential issues with the vehicle?
2. What are the possible reasons for the low predictive score?
3. What could be the thoughts and feelings of the vehicle owner if they get this advice?

<|Answer|>
1. The potential issues with the vehicle could be:
- The battery might be discharged or nearing depletion.
- The engine temperature could be too high, indicating a potential problem with the cooling system.
- The oil quality might be poor, suggesting that the engine might be under stress.
- The vibration level could be high, indicating a potential problem with the transmiss

# 4. Output Comparison & Metrics

Here we compare the outputs from Template A and Template B.  
We evaluate them using three metrics:
- **Quality**: clarity and organization of the response.  
- **Detail**: amount of technical or practical information provided.  
- **Relevance**: how closely the advice matches the vehicle data.  


In [ ]:
import pandas as pd

results = [
    {"Template": "Concise (A)", "Output": advice_A},
    {"Template": "Detailed (B)", "Output": advice_B}
]

df_results = pd.DataFrame(results)
display(df_results)

# Simple scoring rubric
def score_output(text):
    quality = 5 if "Provide" in text or "structured" in text else 3
    detail = text.count("-") + text.count("•")
    relevance = 5 if "oil" in text.lower() or "battery" in text.lower() else 3
    return {"Quality": quality, "Detail": min(5, detail), "Relevance": relevance}

metrics = []
for r in results:
    scores = score_output(r["Output"])
    metrics.append({"Template": r["Template"], **scores})

df_metrics = pd.DataFrame(metrics)
display(df_metrics)


,Template,Output
0,Concise (A),\nYou are a maintenance advisor.\nGiven this v...
1,Detailed (B),\nYou are a professional vehicle maintenance e...


,Template,Quality,Detail,Relevance
0,Concise (A),5,4,5
1,Detailed (B),5,5,5


# 5. Final Prompt

After comparing both templates, we design a **hybrid prompt**.  
This final prompt combines:
- The **clarity and structure** of Template B.  
- The **brevity and practicality** of Template A.  


In [ ]:
def final_prompt(row):
    return f"""
You are a professional vehicle maintenance advisor.
Analyze this vehicle and provide structured yet concise advice:

- Make/Model: {row['Make_and_Model']}
- Year: {row['Year_of_Manufacture']}
- Usage Hours: {row['Usage_Hours']}
- Engine Temperature: {row['Engine_Temperature']}
- Oil Quality: {row['Oil_Quality']}
- Vibration Level: {row['Vibration_Levels']}
- Battery Condition: {row['Battery_Status']}
- Predictive Score: {row['Predictive_Score']}

Instructions:
1. Immediate actions (1–2 lines).
2. Preventive steps (bullets).
3. Risk assessment (short).
4. Next service recommendation (concise).
"""


#  Justification

- **Template A** is concise and easy to read but lacks depth.  
- **Template B** is structured and detailed but sometimes verbose.  
- The **Final Prompt (Hybrid)** balances both: it enforces structure while keeping the advice short and practical.  
- This makes it suitable for real-world deployment where clarity and usability are critical.  


#  Final Output

Finally, we generate maintenance advice using the **Final Prompt**.  
This demonstrates the chosen template in action and validates its effectiveness.  


In [ ]:
final_advice = generator(final_prompt(sample_row), max_new_tokens=200, temperature=0.3)[0]['generated_text']
print("=== FINAL PROMPT OUTPUT ===\n", final_advice)


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


=== FINAL PROMPT OUTPUT ===
 
You are a professional vehicle maintenance advisor.
Analyze this vehicle and provide structured yet concise advice:

- Make/Model: ford f150
- Year: 1.0
- Usage Hours: 0.0189461642954171
- Engine Temperature: 0.0
- Oil Quality: 0.682216289264067
- Vibration Level: 0.0509541472633946
- Battery Condition: 0.0
- Predictive Score: 0.2434626112646807

Instructions:
1. Immediate actions (1–2 lines).
2. Preventive steps (bullets).
3. Risk assessment (short).
4. Next service recommendation (concise).

Question: What is the complete advice you would give to the vehicle owner?



Immediate actions:
- Check the engine oil level and top it up if necessary.
- Ensure the battery is fully charged.

Preventive steps:
- Regularly check the engine temperature to avoid overheating.
- Monitor the oil quality and change it as per the manufacturer's recommendation.
- Check the vibration level and address any issues promptly.
- Regularly inspect the battery and replace it if nec

# 6. Metrics Evaluation for Final Prompt

We now evaluate the **Final Prompt (Hybrid)** output using the same rubric (0–5 scale):

- **Quality**: Measures clarity and organization.  
- **Detail**: Measures amount of actionable information (bullets, sections, specifics).  
- **Relevance**: Measures how closely the advice matches the vehicle data.  

This ensures consistency in comparing the Final Prompt against Template A and Template B.


In [ ]:
# Evaluate the Final Prompt output
final_metrics = score_output(final_advice)

df_final_metrics = pd.DataFrame([{
    "Template": "Final Prompt (Hybrid)",
    **final_metrics
}])

display(df_final_metrics)


,Template,Quality,Detail,Relevance
0,Final Prompt (Hybrid),5,5,5


#  Comparison of All Templates

We tested three prompt designs using the same vehicle data (Ford F150).  
The table below highlights the strengths, weaknesses, and performance metrics of each template:

| Template                 | Strengths                                                                                      | Weaknesses                                                           | Metrics (Quality / Detail / Relevance) |
|--------------------------|------------------------------------------------------------------------------------------------|----------------------------------------------------------------------|----------------------------------------|
| **Template A (Concise)** | - Clear and brief<br>- Easy to scan quickly                                                    | - Lacks depth<br>- Missing actionable steps                          | 5 / 0 / 5                              |
| **Template B (Detailed)**| - Well-structured sections (Immediate, Preventive, Risk, Next Service)<br>- Rich in detail      | - Verbose<br>- May include redundant checks                          | 5 / 5 / 5                              |
| **Final Prompt (Hybrid)**| - Balanced: structured yet concise<br>- Practical and deployment-ready<br>- Actionable advice  | - Slightly less depth than Template B                                | 5 / 5 / 5                              |

**Conclusion:**  
Template A is fast and readable but lacks depth.  
Template B is comprehensive but may overwhelm with verbosity.  
The Final Prompt (Hybrid) merges the strengths of both ( clarity and structure ) making it the most suitable for real-world use.


# 7. Execution with Microsoft Phi-2

To ensure consistency, we also executed **Template A (Concise)** and **Template B (Detailed)** using **Model 2 (microsoft/phi-2)** on the same vehicle data.  
This allows us to compare outputs and metrics across both models (TinyLlama vs Phi-2).

The outputs below demonstrate how Phi-2 responds to the same templates, enabling a fair evaluation of clarity, detail, and relevance.


In [ ]:
# Execution: Run both templates on a sample vehicle using Microsoft Phi-2
sample_row = df.iloc[0]  # Example vehicle (same as TinyLlama execution)

advice_A_phi2 = generator(template_concise(sample_row), max_new_tokens=150, temperature=0.4)[0]['generated_text']
advice_B_phi2 = generator(template_detailed(sample_row), max_new_tokens=250, temperature=0.4)[0]['generated_text']

print("=== TEMPLATE A (Concise, Phi-2) OUTPUT ===\n", advice_A_phi2)
print("\n=== TEMPLATE B (Detailed, Phi-2) OUTPUT ===\n", advice_B_phi2)

# Evaluate outputs with the same scoring rubric
results_phi2 = [
    {"Template": "Concise (A, Phi-2)", "Output": advice_A_phi2},
    {"Template": "Detailed (B, Phi-2)", "Output": advice_B_phi2}
]

df_results_phi2 = pd.DataFrame(results_phi2)
display(df_results_phi2)

metrics_phi2 = []
for r in results_phi2:
    scores = score_output(r["Output"])
    metrics_phi2.append({"Template": r["Template"], **scores})

df_metrics_phi2 = pd.DataFrame(metrics_phi2)
display(df_metrics_phi2)


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


=== TEMPLATE A (Concise, Phi-2) OUTPUT ===
 
You are a maintenance advisor.
Given this vehicle data:

Make/Model: ford f150
Year: 1.0
Usage Hours: 0.0189461642954171
Engine Temperature: 0.0
Oil Quality: 0.682216289264067
Vibration Level: 0.0509541472633946
Battery Status: 0.0
Predictive Score: 0.2434626112646807

Provide 1–2 lines of concise and practical maintenance advice.

**Solution:**

The Ford F150 is a popular truck model that is known for its durability. However, it's important to remember that even the most durable vehicles require regular maintenance to ensure their longevity.

Given the data, it seems that the vehicle is in good condition. The engine temperature is within the normal range, the oil quality is good, the vibration level is low, and the battery status is fine. However, the usage hours are low, which means that the vehicle is not being used as frequently as it should be. This can lead to issues such as battery drain and wear and tear on the engine.

To ensure tha

,Template,Output
0,"Concise (A, Phi-2)",\nYou are a maintenance advisor.\nGiven this v...
1,"Detailed (B, Phi-2)",\nYou are a professional vehicle maintenance e...


,Template,Quality,Detail,Relevance
0,"Concise (A, Phi-2)",5,0,5
1,"Detailed (B, Phi-2)",5,5,5


# Execution of Hybrid (final) Prompt with Microsoft Phi-2

To validate consistency, we executed the **Hybrid Prompt** using **Model 2 (microsoft/phi-2)** on the same vehicle data.  
This ensures a fair comparison of Hybrid outputs across both models (TinyLlama vs Phi-2).


In [ ]:
# Execution: Hybrid Prompt using Microsoft Phi-2
sample_row = df.iloc[0]  # Same vehicle used for A and B

advice_hybrid_phi2 = generator(final_prompt(sample_row), max_new_tokens=250, temperature=0.4)[0]['generated_text']


print("=== HYBRID PROMPT (Phi-2) OUTPUT ===\n", advice_hybrid_phi2)

# Evaluate Hybrid output with the same scoring rubric
results_hybrid_phi2 = [{"Template": "Hybrid (Phi-2)", "Output": advice_hybrid_phi2}]
df_results_hybrid_phi2 = pd.DataFrame(results_hybrid_phi2)
display(df_results_hybrid_phi2)

metrics_hybrid_phi2 = []
for r in results_hybrid_phi2:
    scores = score_output(r["Output"])
    metrics_hybrid_phi2.append({"Template": r["Template"], **scores})

df_metrics_hybrid_phi2 = pd.DataFrame(metrics_hybrid_phi2)
display(df_metrics_hybrid_phi2)


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


=== HYBRID PROMPT (Phi-2) OUTPUT ===
 
You are a professional vehicle maintenance advisor.
Analyze this vehicle and provide structured yet concise advice:

- Make/Model: ford f150
- Year: 1.0
- Usage Hours: 0.0189461642954171
- Engine Temperature: 0.0
- Oil Quality: 0.682216289264067
- Vibration Level: 0.0509541472633946
- Battery Condition: 0.0
- Predictive Score: 0.2434626112646807

Instructions:
1. Immediate actions (1–2 lines).
2. Preventive steps (bullets).
3. Risk assessment (short).
4. Next service recommendation (concise).

Question: What are the immediate actions, preventive steps, risk assessment, and next service recommendation?



Immediate Actions:
The immediate actions are to check the engine oil level and the battery condition.

Preventive Steps:
The preventive steps are to check the vehicle's usage hours, engine temperature, and vibration level.

Risk Assessment:
The risk assessment is that the vehicle is still under warranty and the battery condition might be a risk fa

,Template,Output
0,Hybrid (Phi-2),\nYou are a professional vehicle maintenance a...


,Template,Quality,Detail,Relevance
0,Hybrid (Phi-2),5,5,5


# Execution Results (Phi-2)

In the final run, we executed **Template A (Concise)**, **Template B (Detailed)**, and the **Hybrid Prompt** using **Microsoft Phi-2** on the same vehicle data.

### Outputs:
- **Concise (A, Phi-2)**:  
  Produced a short, general recommendation. Clear and relevant, but lacked actionable detail.  
  **Metrics:** Quality = 5, Detail = 0, Relevance = 5  

- **Detailed (B, Phi-2)**:  
  Generated structured guidance with Immediate Actions, Preventive Steps, Risk Assessment, and Next Service.  
  **Metrics:** Quality = 5, Detail = 5, Relevance = 5  

- **Hybrid (Phi-2)**:  
  Delivered balanced, structured yet concise advice covering Immediate Actions, Preventive Steps, Risk Assessment, and Next Service Recommendation.  
  **Metrics:** Quality = 5, Detail = 5, Relevance = 5  

### Summary:
- Phi-2 behaved consistently with TinyLlama: Concise template gave minimal advice, Detailed template provided comprehensive recommendations, and Hybrid achieved the best balance.  
- This confirms that the **Hybrid (final) Prompt** is robust across both models, making it the optimal choice for deployment in predictive maintenance systems.

